In [ ]:
import time
import smbus2
import bme280
import RPi.GPIO as GPIO
import paho.mqtt.client as mqtt

# Set up MQTT Broker and Topics. 
mqtt_broker = "test.mosquitto.org"
topic_temp = "SmartHomeGrp1/sensor/temperature"
topic_humidity = "SmartHomeGrp1/sensor/humidity"
topic_pressure = "SmartHomeGrp1/sensor/pressure"
topic_fan = "SmartHomeGrp1/device/fan"

# Set up the GPIO pins for the fan, buzzer and LED
fan_pin = 26
buzzer_pin = 21
LED_pin = 20
GPIO.setmode(GPIO.BCM)
GPIO.setup(fan_pin, GPIO.OUT)
GPIO.setup(buzzer_pin, GPIO.OUT)
GPIO.setup(LED_pin, GPIO.OUT)

# Set up the BME280 sensor
port = 1
address = 0x76
bus = smbus2.SMBus(port)
calibration_params = bme280.load_calibration_params(bus, address)

# Set the temp, humidity and pressure limits
temp_limit = 30
humidity_limit = 70
pressure_limit = 1000

#Try catch method for KeyboardInterrupt exception
try:


    while True:
        # Read the temperature, humidity and pressure data from the BME280 sensor
        data = bme280.sample(bus, address, calibration_params)
        temperature = data.temperature
        humidity = data.humidity
        pressure = data.pressure
        
        #Set Up MQTT Payload
        data_temp = str(data.temperature)
        data_humidity = str(data.humidity)
        data_pressure = str(data.pressure)
        data_time = str(data.timestamp)

        # Print the temperature, humidity and pressure values with respective units
        print("Temperature: {:.2f} °C".format(temperature))
        print("Humidity: {:.2f} %".format(humidity))
        print("Pressure: {:.2f} Pa" .format(pressure))

        # Check if the temperature, humidity or pressure  is above the set limit
        if temperature >= temp_limit:
            GPIO.output(fan_pin, GPIO.HIGH)
            
            GPIO.output(buzzer_pin, GPIO.HIGH)
            time.sleep(1)
            GPIO.output(buzzer_pin, GPIO.LOW)
            time.sleep(1)
        else:
            GPIO.output(fan_pin, GPIO.LOW)
            GPIO.output(buzzer_pin, GPIO.LOW)
        
        if humidity >= humidity_limit:
            GPIO.output(buzzer_pin, GPIO.HIGH)
            time.sleep(1)
            GPIO.output(buzzer_pin, GPIO.LOW)
            time.sleep(1)
        else:
           GPIO.output(buzzer_pin, GPIO.LOW)
        
        if pressure >= pressure_limit:
            GPIO.output(LED_pin, GPIO.HIGH)
            time.sleep(1)
            GPIO.output(LED_pin, GPIO.LOW)
            time.sleep(1)
        else:
            GPIO.output(LED_pin, GPIO.LOW)
            
           # Delay inbetween readings 
        time.sleep(1)
        
        #Creating MQTT
        my_mqtt = mqtt.Client()
        print("\nCreated client object at " + time.strftime("%H:%M:%S"))
        #Connecting to MQTT Broker 
        my_mqtt.connect(mqtt_broker, port=1883)
        print("--Connected to broker")
        
        #Publishing sensor data onto MQTT broker 
        try:
            my_mqtt.publish(topic_temp, data_temp)
            my_mqtt.publish(topic_humidity, data_humidity)
            my_mqtt.publish(topic_pressure, data_pressure)
            my_mqtt.publish(topic_fan)
            
        except:
            print("--Error publishing")
        else:
            my_mqtt.disconnect()
            print("--Disconnected from broker")


        time.sleep(1)
        

#CTRL + C to stop MQTT BME280 Sensor Program
except KeyboardInterrupt:
     print("Keyboard Input detected: Turning off fan, LED and buzzer.")
     GPIO.output(fan_pin, GPIO.LOW)
     GPIO.output(buzzer_pin, GPIO.LOW)
     GPIO.output(LED_pin,GPIO.LOW)
    #Cleanup resets all GPIO pins to default 
     GPIO.cleanup()